In [ ]:
!nvidia-smi

Mon Jul 10 06:11:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U pip
!pip install accelerate==0.20.3
!pip install appdirs==1.4.4
!pip install bitsandbytes==0.39.0
!pip install datasets==2.10.1
!pip install fire==0.5.0
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9
!pip install torch==2.0.0
!pip install sentencepiece==0.1.97
!pip install tensorboardX==2.6
# !pip install gradio==3.23.0

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
model_name = "decapoda-research/llama-7b-hf"
# model_name = "ybelkada/falcon-7b-sharded-bf16"








from utils.prompter import Prompter

add_eos_token: bool = False
cutoff_len = CUTOFF_LEN = 256
train_on_inputs = False
prompt_template_name: str = "alpaca"
prompter = Prompter(prompt_template_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False


from transformers import LlamaTokenizer, LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"



def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result


def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt


from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]
BATCH_SIZE = 128
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 200
OUTPUT_DIR = "experiments-200steps"


config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [ ]:
data_path = "reddit.json"
from datasets import load_dataset
data = load_dataset("json", data_files=data_path)


train_val = data["train"].train_test_split(
    test_size=200, shuffle=True, seed=42
)
train_data = (
    train_val["train"].shuffle().map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].shuffle().map(generate_and_tokenize_prompt)
)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0a413311a839dd14/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/16782 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
model.print_trainable_parameters()
num_epochs = 1
# per_device_train_batch_size*gradient_accumulation_steps should be 16? QLoRA?
# optim="paged_adamw_8bit",
# val_set_size
from transformers import TrainingArguments
training_arguments = TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    # max_steps=TRAIN_STEPS,
    num_train_epochs=num_epochs,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    optim="paged_adamw_8bit",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    save_steps=50,
    output_dir=OUTPUT_DIR,
    save_total_limit=10,
    # lr_scheduler_type = "constant",
    # load_best_model_at_end=True,
    report_to="tensorboard"
)

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)
from transformers import Trainer
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator = data_collator
)
model.config.use_cache = False


model = torch.compile(model)

trainable params: 4,194,304 || all params: 3,504,607,232 || trainable%: 0.11967971650867153


In [ ]:
trainer.train()
model.save_pretrained(OUTPUT_DIR)

Step,Training Loss,Validation Loss
50,1.104300,1.046251
100,0.786700,0.753001


The intermediate checkpoints of PEFT may not be saved correctly, using `TrainerCallback` to save adapter_model.bin in corresponding folders, here are some examples https://github.com/huggingface/peft/issues/96


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1696 in train                    │
│                                                                                                  │
│   1693 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1694 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1695 │   │   )                                                                                 │
│ ❱ 1696 │   │   return inner_training_loop(                                                       │
│   1697 │   │   │   args=args,                                                                    │
│   1698 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1699 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2095 in _inner_training_loop     │
│                                                                                                  │
│   2092 │   │   │   elif is_sagemaker_mp_enabled():                                               │
│   2093 │   │   │   │   smp.barrier()                                                             │
│   2094 │   │   │                                                                                 │
│ ❱ 2095 │   │   │   self._load_best_model()                                                       │
│   2096 │   │                                                                                     │
│   2097 │   │   # add remaining tr_loss                                                           │
│   2098 │   │   self._total_loss_scalar += tr_loss.item()                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2292 in _load_best_model         │
│                                                                                                  │
│   2289 │   │   │   │   │   │   # which takes *args instead of **kwargs                           │
│   2290 │   │   │   │   │   │   load_result = model.load_state_dict(state_dict, False)            │
│   2291 │   │   │   │   if not is_sagemaker_mp_enabled():                                         │
│ ❱ 2292 │   │   │   │   │   self._issue_warnings_after_load(load_result)                          │
│   2293 │   │   elif os.path.exists(os.path.join(self.state.best_model_checkpoint, WEIGHTS_INDEX  │
│   2294 │   │   │   load_result = load_sharded_checkpoint(                                        │
│   2295 │   │   │   │   model, self.state.best_model_checkpoint, strict=is_sagemaker_mp_enabled(  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
UnboundLocalError: local variable 'load_result' referenced before assignment

In [ ]:
model.save_pretrained(OUTPUT_DIR)

In [ ]:
!zip -r experiments-200steps.zip experiments-200steps/

  adding: experiments-200steps/ (stored 0%)
  adding: experiments-200steps/runs/ (stored 0%)
  adding: experiments-200steps/runs/Jul09_20-40-55_c8d733e61349/ (stored 0%)
  adding: experiments-200steps/runs/Jul09_20-40-55_c8d733e61349/events.out.tfevents.1688935422.c8d733e61349.535.0 (deflated 59%)
  adding: experiments-200steps/runs/Jul09_20-40-55_c8d733e61349/1688935422.2536492/ (stored 0%)
  adding: experiments-200steps/runs/Jul09_20-40-55_c8d733e61349/1688935422.2536492/events.out.tfevents.1688935422.c8d733e61349.535.1 (deflated 62%)
  adding: experiments-200steps/adapter_config.json (deflated 42%)
  adding: experiments-200steps/adapter_model.bin (deflated 7%)
  adding: experiments-200steps/checkpoint-100/ (stored 0%)
  adding: experiments-200steps/checkpoint-100/optimizer.pt (deflated 15%)
  adding: experiments-200steps/checkpoint-100/rng_state.pth (deflated 28%)
  adding: experiments-200steps/checkpoint-100/training_args.bin (deflated 49%)
  adding: experiments-200steps/checkpoint

In [ ]:
!zip -r experiments-200steps_runs.zip experiments-200steps/runs

  adding: experiments-200steps/runs/ (stored 0%)
  adding: experiments-200steps/runs/Jul09_20-40-55_c8d733e61349/ (stored 0%)
  adding: experiments-200steps/runs/Jul09_20-40-55_c8d733e61349/events.out.tfevents.1688935422.c8d733e61349.535.0 (deflated 59%)
  adding: experiments-200steps/runs/Jul09_20-40-55_c8d733e61349/1688935422.2536492/ (stored 0%)
  adding: experiments-200steps/runs/Jul09_20-40-55_c8d733e61349/1688935422.2536492/events.out.tfevents.1688935422.c8d733e61349.535.1 (deflated 62%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp experiments-200steps/adapter_model.bin /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b

In [ ]:
!cp experiments-200steps/adapter_config.json /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b

In [ ]:
!cp -r experiments-200steps/runs /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b

In [ ]:
!cp -r experiments-200steps/checkpoint-100/ /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b